#問題1   
CIFAR-10のデータセットに対してより高い精度を実現するプログラム(PyTorch)を作成せよ。ただし、プログラムは第10回の講義資料のプログラム(もしくはLab Work (4)で作成したプログラム)を改良して作成せよ。

In [ ]:
#ライブラリの読み込み
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torchvision as tv
import torch.nn as nn

In [ ]:
#訓練データとテストデータの読み込み(初めて実行するときはデーをネットからダウンロードする)
train_dataset = tv.datasets.CIFAR10(root="./", train=True,transform=tv.transforms.ToTensor(),download=True) 
test_dataset = tv.datasets.CIFAR10(root="./", train=False,transform=tv.transforms.ToTensor(),download=True)
#訓練データとテストデー タのミニバッチ処理 ・ミニバッチサイズ=100 ・データの順番をシャッ フル
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# GPUが利用可能ならDEVICE="cuda"
# CPUを利用するのならDEVICE="cpu"
MODELNAME = 'CIFAR10_2D.model'
EPOCH = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class CIFAR10_2D(torch.nn.Module):
    # __init__()関数でネットワークの要素を定義
    def __init__(self):
        super(CIFAR10_2D, self).__init__()
        num_classes=10

        self.block1_output = nn.Sequential (
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block2_output = nn.Sequential (
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block3_output = nn.Sequential (
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block4_output = nn.Sequential (
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block5_output = nn.Sequential (
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(512, 512),  #512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 32 ),  #4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(32, num_classes),  #4096
        )
    # forward計算のときに forward()が呼ばれる。 同時にネットワークを構築
    def forward(self, x):
        x = self.block1_output(x)
        x = self.block2_output(x)
        x = self.block3_output(x)
        x = self.block4_output(x)
        x = self.block5_output(x)
        #print(x.size())
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
def train_2D():
    model = CIFAR10_2D().to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    for epoch in range(EPOCH):
        loss =0 
        # データをミニバッチサイズに切り出す
        for images, labels in train_loader:
            images = images.view(-1,3,32,32).to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            y = model(images)
            batchloss =F.cross_entropy(y, labels)
            batchloss.backward()
            optimizer.step()
            loss = loss + batchloss.item()
        print('epoch',epoch,': loss',loss)
    #モデルをファイルに保存
    torch.save(model.state_dict(),MODELNAME)

In [ ]:
def test_2D():
    total = len(test_loader.dataset)
    correct = 0
    model = CIFAR10_2D().to('cpu')
    model.load_state_dict(torch.load(MODELNAME))
    model.eval()
    # ファイルに保存したモデ ルをロード
    for images, labels in test_loader:
        images = images.view(-1,3,32,32).to('cpu')
        y = model(images)
        #最大値の次元IDを得る
        pred_labels = y.max(dim=1)[1]
        correct = correct + (pred_labels ==labels).sum()
    print('correct:',correct.item())
    print('total:',total)
    #正解率を計算
    print('accuracy:',(correct.item()/float(total)))

In [ ]:
import time
start = time.time()
train_2D()
print("Completed:", time.time()-start)

epoch 0 : loss 826.9031218290329
epoch 1 : loss 574.3589366674423
epoch 2 : loss 462.65993332862854
epoch 3 : loss 379.2427268624306
epoch 4 : loss 320.86059579253197
epoch 5 : loss 271.0699910223484
epoch 6 : loss 239.40734526515007
epoch 7 : loss 203.13594709336758
epoch 8 : loss 178.1977562904358
epoch 9 : loss 154.8951351121068
Completed: 1891.3484666347504


In [ ]:
test_2D()

correct: 8097
total: 10000
accuracy: 0.8097
